In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import string
%matplotlib inline

slim = tf.contrib.slim

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
sentences = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
pos = [['pronoun', 'verb', 'adjective'],
     ['noun', 'verb', 'adverb', 'adjective'],
     ['noun', 'verb', 'determiner', 'noun', 'preposition', 'adjective', 'noun'],
     ['noun', 'verb', 'adverb', 'adjective', 'verb']]

In [3]:
word_list = []
for elm in sentences:
    word_list += elm
    
word_list = list(set(word_list))
word_list.sort()
word_list = ['<pad>'] + word_list

word_dic = {word: idx for idx, word in enumerate(word_list)}
print(word_dic)

{'a': 2, 'feel': 7, 'difficult': 5, 'fast': 6, '<pad>': 0, 'I': 1, 'hungry': 10, 'framework': 9, 'for': 8, 'is': 11, 'tensorflow': 13, 'learning': 12, 'very': 14, 'deep': 4, 'changing': 3}


In [4]:
pos_list = []
for elm in pos:
    pos_list += elm
pos_list = list(set(pos_list))
pos_list.sort()
pos_list = ['<pad>']+pos_list
print(pos_list)

pos_dic = {pos : idx for idx, pos in enumerate(pos_list)}
pos_dic

['<pad>', 'adjective', 'adverb', 'determiner', 'noun', 'preposition', 'pronoun', 'verb']


{'<pad>': 0,
 'adjective': 1,
 'adverb': 2,
 'determiner': 3,
 'noun': 4,
 'preposition': 5,
 'pronoun': 6,
 'verb': 7}

In [5]:
pos_idx_to_dic = {elm[1]: elm[0] for elm in pos_dic.items()}
pos_idx_to_dic

{0: '<pad>',
 1: 'adjective',
 2: 'adverb',
 3: 'determiner',
 4: 'noun',
 5: 'preposition',
 6: 'pronoun',
 7: 'verb'}

In [6]:
def pad_seq(sequences, max_len, dic):
    seq_len, seq_indices = [], []
    # 여기서 seq는 한 문장
    for seq in sequences:
        seq_len.append(len(seq))
        seq_idx = [dic.get(word) for word in seq]
        seq_idx += (max_len - len(seq_idx)) * [dic.get('<pad>')]
        seq_indices.append(seq_idx)
    return seq_len, seq_indices

In [7]:
max_length = 10
X_length, X_indices = pad_seq(sequences=sentences, max_len=max_length, dic=word_dic)
print(X_length, np.shape(X_indices))

[3, 4, 7, 5] (4, 10)


In [8]:
y = [elm + ['<pad>'] * (max_length-len(elm)) for elm in pos]
y = [list(map(lambda el : pos_dic.get(el), elm)) for elm in y]
print(np.shape(y))

(4, 10)


In [9]:
y

[[6, 7, 1, 0, 0, 0, 0, 0, 0, 0],
 [4, 7, 2, 1, 0, 0, 0, 0, 0, 0],
 [4, 7, 3, 4, 5, 1, 4, 0, 0, 0],
 [4, 7, 2, 1, 7, 0, 0, 0, 0, 0]]

In [10]:
class SimPosStackedRNN:
    def __init__(self, X_length, X_indices, y, n_of_classes, hidden_dims, max_len, word_dic):
        
        with tf.variable_scope('rnn_input'):
            self._X_length = X_length
            self._X_indices = X_indices
            self._y = y
            self._keep_prob = tf.placeholder(dtype=tf.float32)
            
            one_hot = tf.eye(len(word_dic), dtype=tf.float32)
            self._one_hot = tf.get_variable(name='one_hot_embedding', initializer=one_hot,
                                           trainable=False)
            self._X_batch = tf.nn.embedding_lookup(params=self._one_hot, ids=self._X_indices)
            
        with tf.variable_scope('stacked_rnn'):
            rnn_cells = []
            for hidden_dim in hidden_dims:
                rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_dim,
                                                      activation=tf.nn.tanh)
                rnn_cell = tf.nn.rnn_cell.DropoutWrapper(cell=rnn_cell, output_keep_prob=self._keep_prob)
                rnn_cells.append(rnn_cell)
            else:
                multi_cell = tf.nn.rnn_cell.MultiRNNCell(cells=rnn_cells)
                
            # [batch_size x n_step x n_of_classes]
            self.score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell=multi_cell, output_size=n_of_classes)
            self.outputs, self.states = tf.nn.dynamic_rnn(cell=self.score_cell, 
                                                         inputs=self._X_batch,
                                                         sequence_length=self._X_length,
                                                         dtype=tf.float32)
            
        with tf.variable_scope('rnn_seq2seq_loss'):
            masks = tf.sequence_mask(lengths=self._X_length,maxlen=max_len, dtype=tf.float32)
            self.seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits=self.outputs,
                                                                targets=self._y,
                                                                weights=masks)
            
        with tf.variable_scope('rnn_prediction'):
            self._prediction = tf.argmax(input=self.outputs, axis=2, output_type=tf.int32)
            
    def predict(self, sess, X_length, X_indices, keep_prob=1.):
        feed_prediction={self._X_length:X_length, self._X_indices:X_indices, self._keep_prob:keep_prob}
        return sess.run(self._prediction, feed_dict=feed_prediction)

```python
sim_pos_stacked.outputs
<tf.Tensor 'stacked_rnn/rnn/transpose_1:0' shape=(?, 10, 8) dtype=float32>

sim_pos_stacked.states
(<tf.Tensor 'stacked_rnn/rnn/while/Exit_3:0' shape=(?, 16) dtype=float32>,
 <tf.Tensor 'stacked_rnn/rnn/while/Exit_4:0' shape=(?, 16) dtype=float32>)
```


In [11]:
class SimPosStackedLSTM:
    def __init__(self, X_length, X_indices, y, n_of_classes, hidden_dims, max_len, word_dic):
        
        with tf.variable_scope('lstm_input'):
            self._X_length = X_length
            self._X_indices = X_indices
            self._y = y
            self._keep_prob = tf.placeholder(dtype=tf.float32)
            
            one_hot = tf.eye(len(word_dic), dtype=tf.float32)
            self._one_hot = tf.get_variable(name='one_hot_embedding', initializer=one_hot,
                                           trainable=False)
            self._X_batch = tf.nn.embedding_lookup(params=self._one_hot, ids=self._X_indices)
            
        with tf.variable_scope('stacked_lstm'):
            lstm_cells = []
            for hidden_dim in hidden_dims:
                lstm_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_dim,
                                                       activation=tf.nn.tanh)
                lstm_cell = tf.nn.rnn_cell.DropoutWrapper(cell=lstm_cell,
                                                         output_keep_prob=self._keep_prob)
                lstm_cells.append(lstm_cell)
            else:
                multi_cell = tf.nn.rnn_cell.MultiRNNCell(cells=lstm_cells)
                
            score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell=multi_cell, output_size=n_of_classes)
            self.outputs, self.states = tf.nn.dynamic_rnn(cell=score_cell,
                                                         inputs=self._X_batch,
                                                         sequence_length=self._X_length,
                                                         dtype=tf.float32)
            
        with tf.variable_scope('lstm_seq2seq_loss'):
            masks = tf.sequence_mask(lengths=self._X_length, maxlen=max_len, dtype=tf.float32)
            self.seq2seq_loss = tf.contrib.seq2seq.sequence_loss(logits=self.outputs, targets=self._y,
                                                                weights=masks)
            
        with tf.variable_scope('lstm_prediction'):
            self._prediction = tf.argmax(input=self.outputs, axis=2, output_type=tf.int32)
            
    def predict(self, sess, X_length, X_indices, keep_prob=1.0):
        feed_prediction={self._X_length:X_length, self._X_indices:X_indices, self._keep_prob:keep_prob}
        return sess.run(self._prediction, feed_dict=feed_prediction)                                                         

```python
sim_pos_stacked.outputs
<tf.Tensor 'stacked_lstm/rnn/transpose_1:0' shape=(?, 10, 8) dtype=float32>

sim_pos_stacked.states
(<tf.Tensor 'stacked_lstm/rnn/while/Exit_3:0' shape=(?, 16) dtype=float32>,
 <tf.Tensor 'stacked_lstm/rnn/while/Exit_4:0' shape=(?, 16) dtype=float32>)
```

In [12]:
lr = .003
epochs = 100
batch_size = 2
total_step = int(np.shape(X_indices)[0] / batch_size)
print(total_step)

2


In [13]:
tr_dataset = tf.data.Dataset.from_tensor_slices((X_length, X_indices, y))
tr_dataset = tr_dataset.shuffle(buffer_size = 20)
tr_dataset = tr_dataset.batch(batch_size = batch_size)
tr_iterator = tr_dataset.make_initializable_iterator()
print(tr_dataset)

<BatchDataset shapes: ((?,), (?, 10), (?, 10)), types: (tf.int32, tf.int32, tf.int32)>


In [14]:
X_length_mb, X_indices_mb, y_mb = tr_iterator.get_next()

In [15]:
# sim_pos_stacked = SimPosStackedRNN(X_length=X_length_mb,X_indices=X_indices_mb,y=y_mb,
#                                   n_of_classes=8, hidden_dims=[16,16], max_len=max_length,
#                                   word_dic=word_dic)


sim_pos_stacked = SimPosStackedLSTM(X_length=X_length_mb,X_indices=X_indices_mb,y=y_mb,
                                  n_of_classes=8, hidden_dims=[16,16], max_len=max_length,
                                  word_dic=word_dic)

In [18]:
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = sim_pos_stacked.seq2seq_loss)

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

tr_loss_hist = []

for epoch in range(epochs):
    avg_tr_loss = 0
    tr_step = 0
    
    sess.run(tr_iterator.initializer)
    try:
        while True:
            _, tr_loss = sess.run(fetches = [training_op, sim_pos_stacked.seq2seq_loss],
                                 feed_dict={sim_pos_stacked._keep_prob: 0.5})
            avg_tr_loss += tr_loss
            tr_step += 1
            
    except tf.errors.OutOfRangeError:
        pass
    
    avg_tr_loss /= tr_step
    tr_loss_hist.append(avg_tr_loss)
    if (epoch + 1) % 10 == 0:
        print('epoch : {:3}, tr_loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :  10, tr_loss : 1.899
epoch :  20, tr_loss : 1.803
epoch :  30, tr_loss : 1.167
epoch :  40, tr_loss : 0.949
epoch :  50, tr_loss : 0.749
epoch :  60, tr_loss : 0.763
epoch :  70, tr_loss : 0.661
epoch :  80, tr_loss : 0.584
epoch :  90, tr_loss : 0.601
epoch : 100, tr_loss : 0.526


In [20]:
yhat = sim_pos_stacked.predict(sess = sess, X_length = X_length, X_indices = X_indices)
yhat

array([[6, 7, 1, 0, 0, 0, 0, 0, 0, 0],
       [4, 7, 2, 1, 0, 0, 0, 0, 0, 0],
       [4, 7, 2, 4, 5, 1, 4, 0, 0, 0],
       [4, 7, 2, 1, 7, 0, 0, 0, 0, 0]])

In [21]:
yhat = [list(map(lambda elm : pos_idx_to_dic.get(elm), row)) for row in yhat]
for elm in yhat:
    print(elm)

['pronoun', 'verb', 'adjective', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['noun', 'verb', 'adverb', 'adjective', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['noun', 'verb', 'adverb', 'noun', 'preposition', 'adjective', 'noun', '<pad>', '<pad>', '<pad>']
['noun', 'verb', 'adverb', 'adjective', 'verb', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
